In [1]:
from tabula.io import read_pdf
import pandas as pd
import re
import tabula

#### Função

In [2]:
# Padronzinação de extração de valores para os PDFs 5-9/11-12
def remover_valores(test):
    test = test.iloc[2:,:1]
    test.columns = ['carros_emplacados']

    # retirada dos valores de colocação 
    for column in test.columns:
        test[column] = test[column].apply(lambda x: x[3:] if isinstance(x, str) else x)

    # Aplicando a função lambda para remover os números seguidos de "o" e um espaço, juntamente com os valores a frente
    for column in test.columns:
        test[column] = test[column].apply(lambda x: re.sub(r'\d+o\s.*?(?=\d+o\s|\Z)', '', x) if isinstance(x, str) else x)

    #Captação de valores de emplacamento
    test['quantidade'] = test['carros_emplacados'].apply(lambda x: re.findall(r'\s(\d+(?:\.\d+)*)\s(?!\d)', x)[0] if isinstance(x, str) else None)


    #retirada dos valores de emplacamento da coluna carros_emplacados
    test['carros_emplacados'] = test['carros_emplacados'].apply(lambda x: x.rstrip())
    test['carros_emplacados'] = test['carros_emplacados'].apply(lambda x: ' '.join(x.split(' ')[:-1]))


    return test


#### Captação de dados - PDF

In [3]:
carro = {}
carro_acumulado = {}
marca_acumulado = {}

pdf_nao_lido = []

for i in range(1, 13):

    try:
        pdf_path = f"C:\\Users\\guga1\\Documents\\TCC\\fenabrave\\2022_0{i}_2.pdf"
        
        carro[i] = tabula.io.read_pdf(pdf_path, pages = 6)[0].iloc[:,:3]
        carro_acumulado[i] = tabula.io.read_pdf(pdf_path, pages = 7)[0]

        print(i)
    
    except Exception as e:
        pdf_nao_lido.append(pdf_path)
        print(f'nao foi possivel ler o pdf {i}')



Error importing jpype dependencies. Fallback to subprocess.
No module named 'jpype'


1
2
3
nao foi possivel ler o pdf 4
5
6
7
8
9
nao foi possivel ler o pdf 10
11
12


#### 1º Parte

- Os pdfs 1/4 e 10 possuem uma extração diferente através da biblioteca utilizada. Para resolver este tipo de problema, sera abordado outros tipo de codigo condizente com a resolução de problema estabelecido




In [4]:
dados_carros = pd.DataFrame()
# limpar valor das linhas captadas + abordar apenas dados que importam
for i in range(3,13):

    try: 
        carro[i] = remover_valores(carro[i])
        carro[i]['mes'] = i

        #Adicionando os valores no dataframe
        dados_carros = pd.concat([dados_carros, carro[i]])
    except Exception as e: 
        print(f'nao funcionou{i}')





nao funcionou4
nao funcionou10


#### 2º Parte

-  pdf 1 precisou ser feito a mão; 
-  pdf 4 e 10 não tiveram leitura
-  restante dos pdfs foram possiveis de extrair e modelar da forma correta



In [5]:

dados_carros_acumulados = pd.DataFrame()
# limpar valor das linhas captadas + abordar apenas dados que importam
for i in range(2,13):

    try: 
        carro_acumulado[i] = remover_valores(carro_acumulado[i])
        carro_acumulado[i]['mes'] = i



        dados_carros_acumulados = pd.concat([dados_carros_acumulados, carro_acumulado[i]])
    except Exception as e: 
        print(f'nao funcionou{i}')




nao funcionou4
nao funcionou10


In [6]:
#alteração dos nomes das colunas
dados_carros.columns = ['modelo','quantidade','mes']

dados_carros_acumulados.columns = ['modelo','quantidade','mes']

display(dados_carros_acumulados)
display(dados_carros)

,modelo,quantidade,mes
2,HYUNDAI/HB20,11.795,2
3,GM/ONIX,11.744,2
4,JEEP/COMPASS,9.408,2
5,VW/T CROSS,8.879,2
6,HYUNDAI/CRETA,8.338,2
...,...,...,...
46,BMW/320I,4.909,12
47,FIAT/SIENA,4.141,12
48,MITSUBISHI/ECLIPSE CROSS,3.725,12
49,CAOA CHERY/TIGGO 3X,3.656,12


,modelo,quantidade,mes
2,HYUNDAI/HB20,6.908,3
3,FIAT/MOBI,6.581,3
4,VW/T CROSS,6.549,3
5,GM/ONIX,5.421,3
6,GM/TRACKER,4.748,3
...,...,...,...
46,MITSUBISHI/PAJERO,349,12
47,VOLVO/XC60,312,12
48,VOLVO/XC40,309,12
49,GM/TRAILBLAZER,296,12


#### Parte 3 - Importação dos dados faltantes (pdfs que não foram lidos)

In [7]:
#importação dos dados
carro_faltante = pd.read_csv('fenabrave - pdf faltantes - carro[pdf_faltante].csv')
carro_acumulado_faltante = pd.read_csv('fenabrave - pdf faltantes - carro_acumulado[pdf_faltante].csv')

#### Dataframe Final

In [14]:
#União de dataframe
carro_acumulado = pd.concat([dados_carros_acumulados, carro_acumulado_faltante])
carro = pd.concat([dados_carros, carro_faltante])

#ordenação dos valores de acordo com o mês
carro = carro.sort_values(by = ['mes','quantidade'], ascending =  True)
carro_acumulado = carro_acumulado.sort_values(by = ['mes','quantidade'], ascending =  True)


In [16]:
carro.to_csv('dados_carros_emplacados2022.csv')
carro_acumulado.to_csv('dados_carros_acumulados_emplacados2022.csv')
